#  Generative Adversarial Networks implementation with PyTorch

This projecct is an implementation of General Adversarial Networks (GAN). GANs leverage a two-player game between a generator and a discriminator to learn to generate realistic data samples. The generator learns to produce realistic samples, while the discriminator learns to distinguish between real and fake samples, leading to a competitive learning process that results in increasingly realistic generated samples. This uses the labml package, and is based off of code from labml.

In [1]:
from typing import Any
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data

from labml_helpers.module import Module

## Discriminator Loss

The discriminator has to **ascend** on the gradient,

$$\nabla_{\theta_d} \frac{1}{m} \sum_{i=1}^m \Bigg[
    \log D\Big(\pmb{x}^{(i)}\Big) +
    \log \Big(1 - D\Big(G\Big(\pmb{z}^{(i)}\Big)\Big)\Big)
\Bigg]$$

$m$ is the mini-batch size and $(i)$ is used to index samples in the mini-batch.
$\pmb{x}$ are samples from $p_{data}$ and $\pmb{z}$ are samples from $p_z$.



`logits_true` are logits from $D(\pmb{x}^{(i)})$ and
`logits_false` are logits from $D(G(\pmb{z}^{(i)}))$


In [2]:
class DiscriminatorLogitsLoss(Module):

    def __init__(self, smoothing: float = 0.2):
        super().__init__()
  
        self.loss_true = nn.BCEWithLogitsLoss()
        self.loss_false = nn.BCEWithLogitsLoss()

        # We use label smoothing because it seems to work better in some cases
        self.smoothing = smoothing

        # Labels are registered as buffered and persistence is set to `False`.
        self.register_buffer('labels_true', _create_labels(256, 1.0 - smoothing, 1.0), False)
        self.register_buffer('labels_false', _create_labels(256, 0.0, smoothing), False)

    def forward(self, logits_true: torch.Tensor, logits_false: torch.Tensor):
      
        if len(logits_true) > len(self.labels_true):
            self.register_buffer("labels_true",
                                 _create_labels(len(logits_true), 1.0 - self.smoothing, 1.0, logits_true.device), False)
        if len(logits_false) > len(self.labels_false):
            self.register_buffer("labels_false",
                                 _create_labels(len(logits_false), 0.0, self.smoothing, logits_false.device), False)

        return (self.loss_true(logits_true, self.labels_true[:len(logits_true)]),
                self.loss_false(logits_false, self.labels_false[:len(logits_false)]))

## Generator Loss

Generator should **descend** on the gradient,

$$\nabla_{\theta_g} \frac{1}{m} \sum_{i=1}^m \Bigg[
    \log \Big(1 - D\Big(G\Big(\pmb{z}^{(i)}\Big)\Big)\Big)
\Bigg]$$

In [4]:
class GeneratorLogitsLoss(Module):

    def __init__(self, smoothing: float = 0.2):
        super().__init__()
        self.loss_true = nn.BCEWithLogitsLoss()
        self.smoothing = smoothing
        # We use labels equal to $1$ for $\pmb{x}$ from $p_{G}.$
        # Then descending on this loss is the same as descending on
        # the above gradient.
        self.register_buffer('fake_labels', _create_labels(256, 1.0 - smoothing, 1.0), False)

    def forward(self, logits: torch.Tensor):
        if len(logits) > len(self.fake_labels):
            self.register_buffer("fake_labels",
                                 _create_labels(len(logits), 1.0 - self.smoothing, 1.0, logits.device), False)

        return self.loss_true(logits, self.fake_labels[:len(logits)])

In [5]:
def _create_labels(n: int, r1: float, r2: float, device: torch.device = None):
    return torch.empty(n, 1, requires_grad=False, device=device).uniform_(r1, r2)